In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Carregar os dados
train_df = pd.read_csv(train_file_path, sep='\t', names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', names=["label", "message"])

# Converter labels em números (ham: 0, spam: 1)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Separar os dados de treinamento e os rótulos
train_messages = train_df['message'].values
train_labels = train_df['label'].values
test_messages = test_df['message'].values
test_labels = test_df['label'].values


In [ ]:
# Tokenização e padronização
max_length = 100  # Tamanho máximo das mensagens
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_messages)

# Transformar mensagens em sequências
train_sequences = tokenizer.texts_to_sequences(train_messages)
train_sequences = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(test_messages)
test_sequences = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length)

In [ ]:
# Criar o modelo
model = keras.Sequential([
    layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=16, input_length=max_length),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Saída para classificação binária
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(train_sequences, train_labels, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Função para prever mensagens com base no modelo
def predict_message(pred_text):
    # Transformar a mensagem em sequência
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded_sequence = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length)

    # Fazer a previsão
    prediction = model.predict(padded_sequence)[0][0]

    # Retornar a probabilidade e a etiqueta
    label = 'ham' if prediction < 0.5 else 'spam'
    return [prediction, label]

In [ ]:
# Testar a função e o modelo
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

# Run this cell to test your function and model. Do not modify contents.
test_predictions()


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
